In [6]:
import pandas as pd
import sys
sys.path.append("../")
from src.data.make_dataset import STORM_EVENTS_COMBINED_PATH

In [18]:
combined_files = list(STORM_EVENTS_COMBINED_PATH.glob("*.csv"))
for c_f in combined_files:
    print(c_f.name)

StormEvents_details-all.csv
StormEvents_fatalities-all.csv
StormEvents_locations-all.csv


In [20]:
event_details_df = pd.read_csv(STORM_EVENTS_COMBINED_PATH / "StormEvents_details-all.csv")
print(event_details_df.shape)
event_details_df.head()

F:\Programs\Anaconda3\envs\ExtremeWeather\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (16,25,26,28,29,34,35,37,39,40,42,43,48,49) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(1687710, 51)


,BEGIN_YEARMONTH,BEGIN_DAY,BEGIN_TIME,END_YEARMONTH,END_DAY,END_TIME,EPISODE_ID,EVENT_ID,STATE,STATE_FIPS,...,END_RANGE,END_AZIMUTH,END_LOCATION,BEGIN_LAT,BEGIN_LON,END_LAT,END_LON,EPISODE_NARRATIVE,EVENT_NARRATIVE,DATA_SOURCE
0,195004,28,1445,195004,28,1445,NaN,10096222,OKLAHOMA,40.0,...,0.0,NaN,NaN,35.12,-99.20,35.17,-99.20,NaN,NaN,PUB
1,195004,29,1530,195004,29,1530,NaN,10120412,TEXAS,48.0,...,0.0,NaN,NaN,31.90,-98.60,31.73,-98.60,NaN,NaN,PUB
2,195007,5,1800,195007,5,1800,NaN,10104927,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.58,-75.70,40.65,-75.47,NaN,NaN,PUB
3,195007,5,1830,195007,5,1830,NaN,10104928,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,40.60,-76.75,NaN,NaN,NaN,NaN,PUB
4,195007,24,1440,195007,24,1440,NaN,10104929,PENNSYLVANIA,42.0,...,0.0,NaN,NaN,41.63,-79.68,NaN,NaN,NaN,NaN,PUB
